In [1]:
%cd /content/drive/MyDrive/GSOC/
!pip install onnx
!pip install openvino

!pip install openvino-dev[all]
!pip install nncf
!pip install timm
!mkdir /content/drive/MyDrive/GSOC/model

/content/drive/MyDrive/GSOC
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 54.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 32.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 55.8 MB/s eta 0:00:00
  Created wheel for jstyleson: filename=jstyleson-0.0.2-py3-none-any.whl size=2398 sha256=7338ec74203a07452234fadf889819c9d47c9347514b30ff5b527e8117739e9a
  Stored in directory: /root/.cache/pip/wheels/d4/2a/06/11202ea86be0f51f34e941

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 1.8 MB/s eta 0:00:00
mkdir: cannot create directory ‘/content/drive/MyDrive/GSOC/model’: File exists


#IMPORT LIBRARIES

In [2]:
import logging
import os
import sys
import time
import warnings
import zipfile
import pickle
from pathlib import Path
from typing import List, Tuple

import torch
import torch.nn as nn
import torch.utils.data
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
from nncf import NNCFConfig  # Important - should be imported directly after torch
from nncf.common.logging.logger import set_log_level

set_log_level(logging.ERROR)  # Disables all NNCF info and warning messages
from nncf.torch import create_compressed_model, register_default_init_args
from openvino.runtime import Core
from openvino.tools import mo
from openvino.runtime import serialize
from torch.jit import TracerWarning
from copy import deepcopy
import timm

sys.path.append("/content/drive/MyDrive/GSOC/openvino_notebooks-main/notebooks/utils")
from notebook_utils import download_file

/usr/local/lib/python3.9/dist-packages/openvino/offline_transformations/__init__.py:10: FutureWarning: The module is private and following namespace `offline_transformations` will be removed in the future, use `openvino.runtime.passes` instead!
  warnings.warn(


INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, tensorflow, onnx, openvino


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [3]:
# swin_small_patch4_window7_224
# vit_tiny_patch16_224
# print(device)

#configs for the notebook

In [4]:
IMAGE_SIZE = (64, 64)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DATASET_DIR = Path("/content/dataset")
MODEL_OUTPUT_DIR=Path("/content/drive/MyDrive/GSOC/model")
BASE_MODEL_NAME='vit_tiny_patch16_224'


fp32_checkpoint_filename = Path(MODEL_OUTPUT_DIR/( BASE_MODEL_NAME + "_fp32")).with_suffix(".pth")

fp32_onnx_path = Path(MODEL_OUTPUT_DIR / (BASE_MODEL_NAME + "_fp32")).with_suffix(".onnx")
fp32_ir_path = fp32_onnx_path.with_suffix(".xml")

int8_onnx_path = Path(MODEL_OUTPUT_DIR / (BASE_MODEL_NAME + "_int8")).with_suffix(".onnx")
int8_ir_path = int8_onnx_path.with_suffix(".xml")

int4_onnx_path= Path(MODEL_OUTPUT_DIR / (BASE_MODEL_NAME + "_int4")).with_suffix(".onnx")
int4_ir_path = int4_onnx_path.with_suffix(".xml")
# !mkdir $str(MODEL_OUTPUT_DIR)

# Dataset and Dataloader

In [5]:
def download_tiny_imagenet_200(
    output_dir: Path,
    url: str = "http://cs231n.stanford.edu/tiny-imagenet-200.zip",
    tarname: str = "tiny-imagenet-200.zip",):
    archive_path = output_dir / tarname
    download_file(url, directory=output_dir, filename=tarname)
    zip_ref = zipfile.ZipFile(archive_path, "r")
    zip_ref.extractall(path=output_dir)
    zip_ref.close()
    print(f"Successfully downloaded and extracted dataset to: {output_dir}")


def create_validation_dir(dataset_dir: Path):
    VALID_DIR = dataset_dir / "val"
    val_img_dir = VALID_DIR / "images"

    fp = open(VALID_DIR / "val_annotations.txt", "r")
    data = fp.readlines()

    val_img_dict = {}
    for line in data:
        words = line.split("\t")
        val_img_dict[words[0]] = words[1]
    fp.close()

    for img, folder in val_img_dict.items():
        newpath = val_img_dir / folder
        if not newpath.exists():
            os.makedirs(newpath)
        if (val_img_dir / img).exists():
            os.rename(val_img_dir / img, newpath / img)

if not DATASET_DIR.exists():
    download_tiny_imagenet_200(DATASET_DIR)
    create_validation_dir(Path('/content/dataset/tiny-imagenet-200'))

/content/dataset/tiny-imagenet-200.zip:   0%|          | 0.00/237M [00:00<?, ?B/s]

Successfully downloaded and extracted dataset to: /content/dataset


In [6]:
def create_dataloaders(batch_size: int = 128):
    """Creates train dataloader that is used for quantization initialization and validation dataloader for computing the model accruacy"""
    train_dir = DATASET_DIR /'tiny-imagenet-200' /"train"
    val_dir = DATASET_DIR /'tiny-imagenet-200'/ "val" /'images'
    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
    )
    train_dataset = datasets.ImageFolder(
        train_dir,
        transforms.Compose(
            [
                transforms.Resize(IMAGE_SIZE),
                transforms.ToTensor(),
                normalize,
            ]
        ),
    )
    val_dataset = datasets.ImageFolder(
        val_dir,
        transforms.Compose(
            [transforms.Resize(IMAGE_SIZE), transforms.ToTensor(), normalize]
        ),
    )

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0,
        pin_memory=True,
        sampler=None,
    )

    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=True,
    )
    return train_loader, val_loader


train_loader, val_loader = create_dataloaders()

#UTILITY functions

In [7]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, name: str, fmt: str = ":f"):
        self.name = name
        self.fmt = fmt
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val: float, n: int = 1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = "{name} {val" + self.fmt + "} ({avg" + self.fmt + "})"
        return fmtstr.format(**self.__dict__)

class ProgressMeter(object):
    """Displays the progress of validation process"""

    def __init__(self, num_batches: int, meters: List[AverageMeter], prefix: str = ""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch: int):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print("\t".join(entries))

    def _get_batch_fmtstr(self, num_batches: int):
        num_digits = len(str(num_batches // 1))
        fmt = "{:" + str(num_digits) + "d}"
        return "[" + fmt + "/" + fmt.format(num_batches) + "]"

def get_model_size(ir_path: str, m_type: str = 'Mb',
                   verbose: bool = True) -> float:
    xml_size = os.path.getsize(ir_path)
    bin_size = os.path.getsize(os.path.splitext(ir_path)[0] + '.bin')
    for t in ['bytes', 'Kb', 'Mb']:
        if m_type == t:
            break
        xml_size /= 1024
        bin_size /= 1024
    model_size = xml_size + bin_size
    if verbose:
        print(f'Model graph (xml):   {xml_size:.3f} Mb')
        print(f'Model weights (bin): {bin_size:.3f} Mb')
        print(f'Model size:          {model_size:.3f} Mb')
    return model_size

def accuracy(output: torch.Tensor, target: torch.Tensor, topk: Tuple[int] = (1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

#Train and Test functions

In [8]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter("Time", ":3.3f")
    losses = AverageMeter("Loss", ":2.3f")
    top1 = AverageMeter("Acc@1", ":2.2f")
    top5 = AverageMeter("Acc@5", ":2.2f")
    progress = ProgressMeter(
        len(train_loader), [batch_time, losses, top1, top5], prefix="Epoch:[{}]".format(epoch)
    )

    # Switch to train mode.
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        images = images.to(device)
        target = target.to(device)

        # Compute output.
        output = model(images)
        loss = criterion(output, target)

        # Measure accuracy and record loss.
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # Compute gradient and do opt step.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Measure elapsed time.
        batch_time.update(time.time() - end)
        end = time.time()

        print_frequency = 50
        if i % print_frequency == 0:
            progress.display(i)

    return model

In [9]:
def validate(val_loader: torch.utils.data.DataLoader, model: torch.nn.Module):
    """Compute the metrics using data from val_loader for the model"""
    batch_time = AverageMeter("Time", ":3.3f")
    top1 = AverageMeter("Acc@1", ":2.2f")
    top5 = AverageMeter("Acc@5", ":2.2f")
    progress = ProgressMeter(len(val_loader), [batch_time, top1, top5], prefix="Test: ")

    # Switch to evaluate mode.
    model.eval()
    model.to(device)

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            images = images.to(device)
            target = target.to(device)

            # Compute the output.
            output = model(images)

            # Measure accuracy and record loss.
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # Measure elapsed time.
            batch_time.update(time.time() - end)
            end = time.time()

            print_frequency = 10
            if i % print_frequency == 0:
                progress.display(i)

        print(
            " * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}".format(top1=top1, top5=top5)
        )
    return top1.avg

In [10]:
def train_the_model(model):
    best_acc1 = 0
    # Training loop.
    init_lr=1e-4

    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=init_lr)
    epochs=3
    for epoch in range(0, epochs):
        # Run a single training epoch.
        model=train(train_loader, model, criterion, optimizer, epoch)

        # Evaluate on validation set.
        acc1 = validate(val_loader, model)

        is_best = acc1 > best_acc1
        best_acc1 = max(acc1, best_acc1)

        if is_best:
            checkpoint = {"state_dict": model.state_dict(), "acc1": acc1}
            # torch.save(checkpoint, fp32_pth_path)
            torch.save( model.state_dict(), fp32_checkpoint_filename)
    acc1_fp32 = best_acc1
    print(f"Accuracy of FP32 model: {acc1_fp32:.3f}")

In [ ]:
# model=timm.create_model(BASE_MODEL_NAME, pretrained=True, num_classes=10,  input_size=(3, 64, 256))

# model(torch.rand((5,3,64,64)).to(device))

In [ ]:
NUM_CLASSES = 200
pretrained=True
model=timm.create_model(BASE_MODEL_NAME, pretrained=True, num_classes=NUM_CLASSES,  img_size=64)
model.to(device)
if not pretrained:
    train_the_model(model)
model.load_state_dict(torch.load(fp32_checkpoint_filename))


<All keys matched successfully>

#Convert to ONNX form

In [ ]:
def export_model_to_onnx(model, fp32_onnx_path):
    dummy_input = torch.randn(1, 3, *IMAGE_SIZE).to(device)
    torch.onnx.export(model, dummy_input, fp32_onnx_path, operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK)
    print(f"FP32 ONNX model was exported to {fp32_onnx_path}.")

# if not fp32_onnx_path.exists():
export_model_to_onnx(model, fp32_onnx_path)

FP32 ONNX model was exported to /content/drive/MyDrive/GSOC/model/vit_tiny_patch16_224_fp32.onnx.


In [ ]:
nncf_config_dict_01 = {
    "input_info": {"sample_size": [1, 3, *IMAGE_SIZE  ]},
    "log_dir": str(MODEL_OUTPUT_DIR),  # The log directory for NNCF-specific logging outputs.
    "target_device":"TRIAL",
    "compression": {
        "algorithm": "quantization",
        "overflow_fix":"enable"}  # Specify the algorithm here.}
}
nncf_config01 = NNCFConfig.from_dict(nncf_config_dict_01)
nncf_config01 = register_default_init_args(
    nncf_config01, 
    deepcopy(train_loader), 
    criterion=nn.CrossEntropyLoss())


In [ ]:
compression_ctrl01, quantized_model = create_compressed_model(model, nncf_config01)
compression_ctrl01.export_model(int8_onnx_path)


In [ ]:
def test_the_quantisation():
    acc1 = validate(val_loader, model)
    print(f"Accuracy of initialized FP32 model: {acc1:.3f}")

    acc2 = validate(val_loader, quantized_model)
    print(f"Accuracy of initialized INT8 model: {acc2:.3f}")

test_the_quantisation()

Test: [ 0/79]	Time 0.132 (0.132)	Acc@1 71.09 (71.09)	Acc@5 87.50 (87.50)
Test: [10/79]	Time 0.136 (0.136)	Acc@1 45.31 (56.46)	Acc@5 80.47 (81.25)
Test: [20/79]	Time 0.155 (0.139)	Acc@1 45.31 (53.61)	Acc@5 73.44 (80.13)
Test: [30/79]	Time 0.152 (0.148)	Acc@1 40.62 (51.81)	Acc@5 64.84 (77.87)
Test: [40/79]	Time 0.173 (0.154)	Acc@1 61.72 (50.67)	Acc@5 82.81 (76.87)
Test: [50/79]	Time 0.193 (0.158)	Acc@1 46.88 (50.00)	Acc@5 77.34 (76.39)
Test: [60/79]	Time 0.175 (0.161)	Acc@1 53.91 (49.18)	Acc@5 81.25 (75.37)
Test: [70/79]	Time 0.167 (0.162)	Acc@1 46.09 (48.61)	Acc@5 75.00 (75.02)
 * Acc@1 49.730 Acc@5 75.970
Accuracy of initialized FP32 model: 49.730
Test: [ 0/79]	Time 0.291 (0.291)	Acc@1 61.72 (61.72)	Acc@5 85.16 (85.16)
Test: [10/79]	Time 0.260 (0.287)	Acc@1 47.66 (52.49)	Acc@5 75.78 (77.34)
Test: [20/79]	Time 0.261 (0.285)	Acc@1 42.19 (49.55)	Acc@5 65.62 (76.23)
Test: [30/79]	Time 0.288 (0.287)	Acc@1 41.41 (47.25)	Acc@5 60.16 (74.47)
Test: [40/79]	Time 0.428 (0.310)	Acc@1 57.81 (46.34)

In [ ]:
def create_bin_and_xml_files(onnx_model_path, ir_path):
    onnx_model = mo.convert_model(str(onnx_model_path))
    serialize(model=onnx_model, xml_path=str(ir_path))
    
create_bin_and_xml_files(int8_onnx_path, int8_ir_path)
create_bin_and_xml_files(fp32_onnx_path, fp32_ir_path)

In [ ]:
print(get_model_size(int8_ir_path, verbose=True))
print(get_model_size(fp32_ir_path))

Model graph (xml):   0.673 Mb
Model weights (bin): 5.371 Mb
Model size:          6.044 Mb
6.044435501098633
Model graph (xml):   0.317 Mb
Model weights (bin): 21.090 Mb
Model size:          21.406 Mb
21.406378746032715


In [ ]:
def parse_benchmark_output(benchmark_output: str):
    """Prints the output from benchmark_app in human-readable format"""
    parsed_output = [line for line in benchmark_output if 'FPS' in line]
    print(*parsed_output, sep='\n')

print('Benchmark FP32 model (OpenVINO IR)')
benchmark_output = ! benchmark_app -m "$fp32_ir_path" -d CPU -api async -t 15
parse_benchmark_output(benchmark_output)

print('Benchmark INT8 model (OpenVINO IR)')
benchmark_output = ! benchmark_app -m "$int8_ir_path" -d CPU -api async -t 15
parse_benchmark_output(benchmark_output)

Benchmark FP32 model (OpenVINO IR)
[ INFO ] Throughput:   164.38 FPS
Benchmark INT8 model (OpenVINO IR)
[ INFO ] Throughput:   188.12 FPS


In [ ]:

# https://openvinotoolkit.github.io/nncf/

nncf_config_dict02={
    "input_info": { "sample_size": [1, 3,*IMAGE_SIZE] },

    "compression": {
       "algorithm": "quantization",
       "weights":{"bits":4},
       "activations":{"bits":8},
       "overflow_fix":"enable"
       },
    "target_device": "TRIAL"
}


criterion = nn.CrossEntropyLoss().to(device)
nncf_config02 = NNCFConfig.from_dict(nncf_config_dict02)


nncf_config02 = register_default_init_args(nncf_config02, (train_loader),
                                    criterion=criterion)
compression_ctrl, four_bit_quantized_model = create_compressed_model(deepcopy(model), nncf_config02)
compression_ctrl.export_model(int4_onnx_path)

In [ ]:
acc3 = validate(val_loader, four_bit_quantized_model)
print(f"Accuracy of initialized INT8 model: {acc3:.3f}")

Test: [ 0/79]	Time 0.522 (0.522)	Acc@1 0.00 (0.00)	Acc@5 0.78 (0.78)


In [ ]:
print('Benchmark int4 model (OpenVINO IR)')
create_bin_and_xml_files(int4_onnx_path, int4_ir_path)

benchmark_output = ! benchmark_app -m "$int4_ir_path" -d CPU -api async -t 15
parse_benchmark_output(benchmark_output)

Benchmark int4 model (OpenVINO IR)
[ INFO ] Throughput:   151.93 FPS


In [ ]:

print(get_model_size(int4_ir_path))
print(get_model_size(int8_ir_path))
print(get_model_size(fp32_ir_path))

# int4_onnx_path= Path(MODEL_OUTPUT_DIR / (BASE_MODEL_NAME + "_int8")).with_suffix(".onnx")
# int4_ir_path = int4_onnx_path.with_suffix(".xml")

Model graph (xml):   0.713 Mb
Model weights (bin): 2.751 Mb
Model size:          3.465 Mb
3.464742660522461
Model graph (xml):   0.674 Mb
Model weights (bin): 5.371 Mb
Model size:          6.045 Mb
6.045186996459961
Model graph (xml):   0.317 Mb
Model weights (bin): 21.090 Mb
Model size:          21.407 Mb
21.406603813171387


In [ ]:
def fine_tune_the_model(model):
    lr=1e-5
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_acc1 = 0
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    epochs=3
    for epoch in range(0, epochs):
        # Run a single training epoch.
        model=train(train_loader, model, criterion, optimizer, epoch)

        # Evaluate on validation set.
        acc1 = validate(val_loader, model)

        is_best = acc1 > best_acc1
        best_acc1 = max(acc1, best_acc1)

        # if is_best:
        #     checkpoint = {"state_dict": model.state_dict(), "acc1": acc1}
        #     # torch.save(checkpoint, fp32_pth_path)
        #     torch.save( model.state_dict(), fp32_checkpoint_filename)
        print(best_acc1)
    acc1_fp32 = best_acc1
    return model
# four_bit_quantized_model=fine_tune_the_model(four_bit_quantized_model)

In [ ]:
four_bit_quantized_model.parameters

In [ ]:
int4_ir_path

PosixPath('/content/drive/MyDrive/GSOC/model/vit_tiny_patch16_224_int4.xml')

In [ ]:
int8_ir_path

PosixPath('/content/drive/MyDrive/GSOC/model/vit_tiny_patch16_224_int8.xml')